# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [30]:
import requests
import time
import re

from random import randint

In [31]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [32]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    def wrapper(*args, **kwargs):
        start_time = time.time()  # Засекаем время до выполнения функции
        result = func(*args, **kwargs)  # Выполняем функцию
        end_time = time.time()  # Засекаем время после выполнения функции
        elapsed_time = end_time - start_time  # Считаем разницу
        print(f"Время выполнения функции '{func.__name__}': {elapsed_time}")
        return result

    return wrapper

## Задание 2

In [33]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    def wrapper(*args, **kwargs):
        print(f"Функция '{func.__name__}' вызвана с параметрами:")
        print(f"{args} {kwargs}")
        result = func(*args, **kwargs)  # Выполнение функции
        return result

    return wrapper

## Задание 3

In [35]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    def wrapper(*args, **kwargs):
        wrapper.call_count += 1  # Увеличиваем счётчик при каждом вызове
        print(f"Функция '{func.__name__}' была вызвана {wrapper.call_count} раз.")
        return func(*args, **kwargs)  # Выполняем декорируемую функцию

    wrapper.call_count = 0  # Привязываем счётчик вызовов к функции-обёртке
    return wrapper

## Задание 4

In [51]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}
  def fmemo(*args):
      # Если результат для данных аргументов уже сохранён, возвращаем его
      if args in cache:
          print(f"Используем кэш для {args}")
          return cache[args]
      # Иначе вычисляем результат, сохраняем в кэш и возвращаем
      result = func(*args)
      cache[args] = result
      print(f"Рассчитан и сохранён результат для {args}")
      return result

  fmemo.cache = cache  # Привязываем кэш к функции
  return fmemo

## Тестирование

In [37]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция 'wrapper' была вызвана 1 раз.
Функция 'wrapper' вызвана с параметрами:
('whole',) {}
Время выполнения функции 'word_count': 0.317124605178833
Cлово whole встречается 176 раз


In [49]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [50]:
# измеряем время выполнения
# декоратор @benchmark не используем, будет большой лог на каждый рекурсивный вызов
start_time = time.time()  # Засекаем время до выполнения функции
print(f"Результат без хэширования: {fib(35)}")
end_time = time.time()  # Засекаем время после выполнения функции
elapsed_time = end_time - start_time  # Считаем разницу
print(f"Время выполнения функции {elapsed_time}")

Результат без хэширования: 9227465
Время выполнения функции 4.109407901763916


In [47]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [48]:
# измеряем время выполнения
# декоратор @benchmark не используем, будет большой лог на каждый рекурсивный вызов
# также отключен вывод у декоратора @memo, чтобы ресурсы не уходили на вывод в консоль
start_time = time.time()  # Засекаем время до выполнения функции
print(f"Результат с хэшированием: {fib(35)}")
end_time = time.time()  # Засекаем время после выполнения функции
elapsed_time = end_time - start_time  # Считаем разницу
print(f"Функция Время выполнения функции {elapsed_time}")

Результат с хэшированием: 9227465
Функция Время выполнения функции 0.0010967254638671875


**Время выполнения fib(35)**

  без хэширования 4.109407901763916

  с хэшированием  0.0010967254638671875